<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing-Images" data-toc-modified-id="Preprocessing-Images-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing Images</a></span></li><li><span><a href="#GANs:-Discrimator-and-Generator" data-toc-modified-id="GANs:-Discrimator-and-Generator-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GANs: Discrimator and Generator</a></span></li></ul></div>

# GANs for Anime Faces

The main idea of this notebook is to use GANs in order to generate fake anime faces. I would use the tutorial [GANs from Scratch 1: A deep introduction. With code in PyTorch and TensorFlow](https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f).

By the end of this little project, I hope to have some realistic anime faces.

In [1]:
#Loading the required libraries
import torch
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

import glob
from PIL import Image
# from utils import Logger

In [29]:
#Define the required functions
def images_to_vectors(images, value):
    return images.view(images.size(0), value)

def vectors_to_images(vectors, rows, cols):
    return vectors.view(vectors.size(0), 1, rows, cols)

def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, 100))
    return n

## Preprocessing Images

This section contains the reading and preprocessing of the anime faces obtained in [Anime Faces](https://www.kaggle.com/soumikrakshit/anime-faces/data) and the ones thanks to [Brian Chao](https://github.com/Mckinsey666) and his repostory [Anime-Face-Dataset](https://github.com/Mckinsey666/Anime-Face-Dataset). 

In [2]:
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        ])
    out_dir = './dataset'
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)
# Load data
data = mnist_data()
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
# Num batches
num_batches = len(data_loader)

In [23]:
#Create transformers image to tensor and tensor to image 
trans_0 = transforms.ToPILImage()
trans_1 = transforms.ToTensor()
trans_2 = transforms.Normalize(mean=(0, 0, 0), std=(1, 1, 1))

In [24]:
#Read the images and createds a list with the images in tensor format
images_path = glob.glob("../Images/*png")
images_data = []
no_charged = 0
for idx, image in enumerate(images_path):
    try:
        img = Image.open(image)
        img_tensor = trans_2(trans_1(img))
        images_data.append(img_tensor)
    except:
        no_charged += 1
        pass
    if idx % 10000 == 0 and idx > 0:
        print('IMAGE NUMBER', idx)
#Print the number of images non charged
print('NON CHARGED', no_charged, 'IMAGES')

IMAGE NUMBER 10000
IMAGE NUMBER 20000
NON CHARGED 0 IMAGES


In [25]:
#Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(images_data, batch_size=100, shuffle=True)
num_batches = len(data_loader)

In [26]:
pixel_rows = images_data[0].shape[1]
pixel_cols = images_data[0].shape[2]
print(pixel_rows, pixel_cols)

64 64


## GANs: Discrimator and Generator

In this section we define the architecture of the two neural networks: the discriminator and the generator using the code in [GANs from Scratch 1: A deep introduction. With code in PyTorch and TensorFlow](https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f).

In [27]:
class DiscriminatorNet(torch.nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = pixel_rows*pixel_cols
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
discriminator = DiscriminatorNet()

In [31]:
class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 100
        n_out = pixel_rows*pixel_cols
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
generator = GeneratorNet()

In [33]:
#Define the optimizers and lossfor the NNs
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

loss = nn.BCELoss()